In [1]:
import pandas as pd

pandasでtidyverseのgather/spreadをやりたい。今どきはpivot_longer/widerというらしい

In [14]:
df = pd.DataFrame(dict(
    x=[1,2,3],
    y=[4,5,6]
), index=[0,1,2])

In [15]:
df

,x,y
0,1,4
1,2,5
2,3,6


## pivot_longer

### stack

### stackの説明
Signature: df.stack(level=-1, dropna=True)

Stack the prescribed level(s) from columns to index.

Return a reshaped DataFrame or Series having a multi-level
index with one or more new inner-most levels compared to the current
DataFrame. The new inner-most levels are created by pivoting the
columns of the current dataframe:

  - if the columns have a single level, the output is a Series;
  - if the columns have multiple levels, the new index
    level(s) is (are) taken from the prescribed level(s) and
    the output is a DataFrame.

The new index levels are sorted.

#### Parameters
level : int, str, list, default -1
    Level(s) to stack from the column axis onto the index
    axis, defined as one index or label, or a list of indices
    or labels.

dropna : bool, default True
    Whether to drop rows in the resulting Frame/Series with
    missing values. Stacking a column level onto the index
    axis can create combinations of index and column values
    that are missing from the original dataframe. See Examples
    section.

In [6]:
df.stack()

0  x    1
   y    4
1  x    2
   y    5
2  x    3
   y    6
dtype: int64

In [16]:
df.stack().reset_index()

,level_0,level_1,0
0,0,x,1
1,0,y,4
2,1,x,2
3,1,y,5
4,2,x,3
5,2,y,6


### pivot_wide_to_long

#### Signature:
pd.wide_to_long(
    df: pandas.core.frame.DataFrame,
    stubnames,
    i,
    j,
    sep: str = '',
    suffix: str = '\\d+',
) -> pandas.core.frame.DataFrame

#### Docstring:
Wide panel to long format. Less flexible but more user-friendly than melt.

With stubnames ['A', 'B'], this function expects to find one or more
group of columns with format
A-suffix1, A-suffix2,..., B-suffix1, B-suffix2,...
You specify what you want to call this suffix in the resulting long format
with `j` (for example `j='year'`)

Each row of these wide variables are assumed to be uniquely identified by
`i` (can be a single column name or a list of column names)

All remaining variables in the data frame are left intact.

#### Parameters
- df : DataFrame
    The wide-format DataFrame.
- stubnames : str or list-like
    The stub name(s). The wide format variables are assumed to
    start with the stub names.
- i : str or list-like
    Column(s) to use as id variable(s).
- j : str
    The name of the sub-observation variable. What you wish to name your
    suffix in the long format.
- sep : str, default ""
    A character indicating the separation of the variable names
    in the wide format, to be stripped from the names in the long format.
    For example, if your column names are A-suffix1, A-suffix2, you
    can strip the hyphen by specifying `sep='-'`.
- suffix : str, default '\\d+'
    A regular expression capturing the wanted suffixes. '\\d+' captures
    numeric suffixes. Suffixes with no numbers could be specified with the
    negated character class '\\D+'. You can also further disambiguate
    suffixes, for example, if your wide variables are of the form
    A-one, B-two,.., and you have an unrelated column A-rating, you can
    ignore the last one by specifying `suffix='(!?one|two)'`.

    .. versionchanged:: 0.23.0
        When all suffixes are numeric, they are cast to int64/float64.

#### Returns
DataFrame
    A DataFrame that contains each stub name as a variable, with new index
    (i, j).

#### Notes
All extra variables are left untouched. This simply uses
`pandas.melt` under the hood, but is hard-coded to "do the right thing"
in a typical case.

列名が、特定の形になってないと使えない・・・？

### melt

### meltの説明
#### Signature:
pd.melt(
    frame: pandas.core.frame.DataFrame,
    id_vars=None,
    value_vars=None,
    var_name=None,
    value_name='value',
    col_level=None,
) -> pandas.core.frame.DataFrame

#### Docstring:
Unpivot a DataFrame from wide to long format, optionally leaving identifiers set.

This function is useful to massage a DataFrame into a format where one
or more columns are identifier variables (`id_vars`), while all other
columns, considered measured variables (`value_vars`), are "unpivoted" to
the row axis, leaving just two non-identifier columns, 'variable' and
'value'.

#### Parameters
- id_vars : tuple, list, or ndarray, optional
    Column(s) to use as identifier variables.
- value_vars : tuple, list, or ndarray, optional
    Column(s) to unpivot. If not specified, uses all columns that
    are not set as `id_vars`.
- var_name : scalar
    Name to use for the 'variable' column. If None it uses
    ``frame.columns.name`` or 'variable'.
- value_name : scalar, default 'value'
    Name to use for the 'value' column.
- col_level : int or str, optional
    If columns are a MultiIndex then use this level to melt.

#### Returns
DataFrame
    Unpivoted DataFrame.


In [17]:
pd.melt(df)

,variable,value
0,x,1
1,x,2
2,x,3
3,y,4
4,y,5
5,y,6


元のindexの情報が失われている。

> one or more columns are identifier variables (id_vars),

と言っているので、dimensionがindexではなくて列になっていることが想定されている？

In [20]:
df2 = df.copy()
df2["idx"] = df2.index
df2.melt(id_vars="idx", value_vars=["x", "y"])

,idx,variable,value
0,0,x,1
1,1,x,2
2,2,x,3
3,0,y,4
4,1,y,5
5,2,y,6


これが、やりたいことと直感に一番合っていると思われる